In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


PATH='/home/wk/myProjects/data/Enron/oper/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

bs=32; bptt=75
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

learner.load_encoder('adam3_10_enc')

In [2]:
#PATH=Path('/home/wk/myProjects/data/Enron/tag/')

In [3]:
#[str(i) for i in PATH.iterdir()]
#PATH/'here.txt'.open()

In [4]:
### Test
ss=r"""
Message-ID: <20945519.1075855715950.JavaMail.evans@thyme>
Date: Tue, 6 Mar 2001 02:55:00 -0800 (PST)
From: phillip.allen@enron.com
To: al.pollard@enron.com
Subject: Re: MS 150
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Al Pollard
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_June2001\Notes Folders\Sent
X-Origin: Allen-P
X-FileName: pallen.nsf

Al,

I was not going to do the MS this year.  Thanks for the offer though.

All is well here.  We went to Colorado last week and the kids learned to 
ski.  Work is same as always.
How are things going at New Power?  Is there any potential?

Phillip



"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

'message-id: <20945519.1075855715950.javamail.evans@thyme> date: tue, 6 mar 2001 02:55:00 -0800 (pst) from: phillip.allen@enron.com to: al.pollard@enron.com subject: re: ms 150 mime-version: 1.0 content-type: text/plain; charset=us-ascii content-transfer-encoding: 7bit x-from: phillip k allen x-to: al pollard x-cc: x-bcc: x-folder: \\phillip_allen_june2001\\notes folders\\sent x-origin: allen-p x-filename: pallen.nsf al, i was not going to do the ms this year. thanks for the offer though. all is well here. we went to colorado last week and the kids learned to ski. work is same as always. how are things going at new power? is there any potential? phillip'

In [5]:
import copy
ss = copy.deepcopy(s)
s_p = [ss[0][0:3]]
t=TEXT.numericalize(s)
#t = t.long().cpu()

m=learner.model

m[0].bs=1
m.eval()
m.reset()

#m.cpu()
#m.cuda()
res,*_ = m(t)

nexts = torch.topk(res[-1], 10)[1]

for i in range(len(s[0])-3):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]    
    if (predict[0:2] == '@@'):
        print("pos=" + str(i+3) + "; " + predict + " <- " + s[0][i+3] , end=' \n')                    
        ss[0][i+3]=predict
    
    s_p = [ss[0][0:i+4]]
    t=TEXT.numericalize(s_p)

    res,*_ = m(t)    
    
m[0].bs=bs       
print('...')

pos=4; @@othr_dt@@ <- 6 
pos=5; @@othr_tm@@ <- mar 
pos=11; @@email@@ <- phillip.allen@enron.com 
pos=13; @@othr_em@@ <- al.pollard@enron.com 
pos=26; @@firstname@@ <- phillip 
pos=27; @@lastname@@ <- k 
...


In [6]:
#https://askubuntu.com/questions/607118/cuda-not-working-after-returning-laptop-from-sleep#750939
len(s[0])

90

In [7]:
#sudo rmmod nvidia_uvm
#sudo modprobe nvidia_uvm

In [8]:
#t.long().cpu()